# NYC Taxi Data Analysis

This project is still in progress, I've uploaded it for feedback and also to show that I'm still around. While this notebook is pretty messy, think of it as thinking outloud. There is still much to do.

1) A pipeline is going to be necessary of course, here, I'm going through the data to see what will be required in that pipeline.

2) (kind of related to 1) Time. Coming up with both a consistant and accurate way to represent time from year to second.

3) As of now I've chosen to work with the entire dataset. This is because I'm looking for errors that could potentially occur. However, for certain algorithms such as SVM, we'd need to scale down severly. This techniqe will implemented in the next upload.

4) Relating to 2, more than likely Ensemble Methods will be required given the complex nature of time, however I will continue my research into this topic for the next upload.

This data was pulled from [TLC](http://www.nyc.gov/html/tlc/html/about/trip_record_data.shtml) (Taxi and Limmousine Commission) in New York City. It contains yellow cab data from June 2019. The goal of this project is to understand the data presented, and use a Machiene Learning Algorithm to look beneath the numbers.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn as sk
import matplotlib.pyplot as plt
import datetime as dt
import time as time

In [2]:
taxi = pd.read_csv('yellow_tripdata_2019-06.csv')
zone = pd.read_csv('taxi+_zone_lookup.csv')

In [3]:
taxi.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2019-06-01 00:55:13,2019-06-01 00:56:17,1,0.0,1,N,145,145,2,3.0,0.5,0.5,0.00,0.0,0.3,4.30,0.0
1,1,2019-06-01 00:06:31,2019-06-01 00:06:52,1,0.0,1,N,262,263,2,2.5,3.0,0.5,0.00,0.0,0.3,6.30,2.5
2,1,2019-06-01 00:17:05,2019-06-01 00:36:38,1,4.4,1,N,74,7,2,17.5,0.5,0.5,0.00,0.0,0.3,18.80,0.0
3,1,2019-06-01 00:59:02,2019-06-01 00:59:12,0,0.8,1,N,145,145,2,2.5,1.0,0.5,0.00,0.0,0.3,4.30,0.0
4,1,2019-06-01 00:03:25,2019-06-01 00:15:42,1,1.7,1,N,113,148,1,9.5,3.0,0.5,2.65,0.0,0.3,15.95,2.5


In [4]:
taxi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6941024 entries, 0 to 6941023
Data columns (total 18 columns):
VendorID                 int64
tpep_pickup_datetime     object
tpep_dropoff_datetime    object
passenger_count          int64
trip_distance            float64
RatecodeID               int64
store_and_fwd_flag       object
PULocationID             int64
DOLocationID             int64
payment_type             int64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
congestion_surcharge     float64
dtypes: float64(9), int64(6), object(3)
memory usage: 953.2+ MB


In [5]:
zone.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


Before moving into the data, lets a look at the zone pick up file to get an idea of what we are looking at.

In [6]:
zone[zone['service_zone']== 'Yellow Zone']

,LocationID,Borough,Zone,service_zone
3,4,Manhattan,Alphabet City,Yellow Zone
11,12,Manhattan,Battery Park,Yellow Zone
12,13,Manhattan,Battery Park City,Yellow Zone
23,24,Manhattan,Bloomingdale,Yellow Zone
42,43,Manhattan,Central Park,Yellow Zone
44,45,Manhattan,Chinatown,Yellow Zone
47,48,Manhattan,Clinton East,Yellow Zone
49,50,Manhattan,Clinton West,Yellow Zone
67,68,Manhattan,East Chelsea,Yellow Zone
78,79,Manhattan,East Village,Yellow Zone


In [7]:
zone[zone['service_zone']=='Boro Zone']

,LocationID,Borough,Zone,service_zone
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
4,5,Staten Island,Arden Heights,Boro Zone
5,6,Staten Island,Arrochar/Fort Wadsworth,Boro Zone
6,7,Queens,Astoria,Boro Zone
7,8,Queens,Astoria Park,Boro Zone
8,9,Queens,Auburndale,Boro Zone
9,10,Queens,Baisley Park,Boro Zone
10,11,Brooklyn,Bath Beach,Boro Zone
13,14,Brooklyn,Bay Ridge,Boro Zone


In [8]:
zone['service_zone'].value_counts()

Boro Zone      205
Yellow Zone     55
Airports         2
EWR              1
Name: service_zone, dtype: int64

So the Taxi zone's are split between Manhattan, all other boroughs, and Newark Airport. Let's start exploring the data and prepare it for our model. First, in addition to the pick up and drop off times, there will be a single column 'Duration' containing the length of the overall trip. Doing this will give us a better idea of the trip, and make computation easier on the algorithm after they are standardized.

There is another column we have to take care of as well, the 'store_and_fwd_flag' column. This column represents whether the trip was stored in the vehicle's memory first, then sent to the vendor. It's values are only 'Y' or 'N' so converting to an integer won't be diffuclt.

In [9]:
pick_up = taxi['tpep_pickup_datetime']
pick_up = pd.to_datetime(pick_up, infer_datetime_format=True)

drop_off = taxi['tpep_dropoff_datetime']
drop_off = pd.to_datetime(drop_off, infer_datetime_format=True)

In [10]:
taxi['duration'] = drop_off - pick_up

In [11]:
taxi.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,duration
0,1,2019-06-01 00:55:13,2019-06-01 00:56:17,1,0.0,1,N,145,145,2,3.0,0.5,0.5,0.00,0.0,0.3,4.30,0.0,00:01:04
1,1,2019-06-01 00:06:31,2019-06-01 00:06:52,1,0.0,1,N,262,263,2,2.5,3.0,0.5,0.00,0.0,0.3,6.30,2.5,00:00:21
2,1,2019-06-01 00:17:05,2019-06-01 00:36:38,1,4.4,1,N,74,7,2,17.5,0.5,0.5,0.00,0.0,0.3,18.80,0.0,00:19:33
3,1,2019-06-01 00:59:02,2019-06-01 00:59:12,0,0.8,1,N,145,145,2,2.5,1.0,0.5,0.00,0.0,0.3,4.30,0.0,00:00:10
4,1,2019-06-01 00:03:25,2019-06-01 00:15:42,1,1.7,1,N,113,148,1,9.5,3.0,0.5,2.65,0.0,0.3,15.95,2.5,00:12:17


In [12]:
taxi['store_and_fwd_flag'].value_counts()

N    6907368
Y      33656
Name: store_and_fwd_flag, dtype: int64

In [13]:
taxi['store_and_fwd_flag'] = taxi['store_and_fwd_flag'].map({'Y':1,'N':0})
taxi.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,duration
0,1,2019-06-01 00:55:13,2019-06-01 00:56:17,1,0.0,1,0,145,145,2,3.0,0.5,0.5,0.00,0.0,0.3,4.30,0.0,00:01:04
1,1,2019-06-01 00:06:31,2019-06-01 00:06:52,1,0.0,1,0,262,263,2,2.5,3.0,0.5,0.00,0.0,0.3,6.30,2.5,00:00:21
2,1,2019-06-01 00:17:05,2019-06-01 00:36:38,1,4.4,1,0,74,7,2,17.5,0.5,0.5,0.00,0.0,0.3,18.80,0.0,00:19:33
3,1,2019-06-01 00:59:02,2019-06-01 00:59:12,0,0.8,1,0,145,145,2,2.5,1.0,0.5,0.00,0.0,0.3,4.30,0.0,00:00:10
4,1,2019-06-01 00:03:25,2019-06-01 00:15:42,1,1.7,1,0,113,148,1,9.5,3.0,0.5,2.65,0.0,0.3,15.95,2.5,00:12:17


In [14]:
times = taxi.iloc[:,[1,2]]

In [15]:
pickup = times['tpep_pickup_datetime']
dropoff = times ['tpep_dropoff_datetime']
pickup = pickup.map(lambda x: x.strip())
dropoff = dropoff.map(lambda x: x.strip())
dropoff = dropoff.map(lambda x: x.strip())

times['tpep_pickup_datetime'] = pickup
times['tpep_dropoff_datetime'] = dropoff
times_dt = times.astype('datetime64[ns]')

/Users/myles/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/myles/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [16]:
from sklearn.preprocessing import MinMaxScaler

def normalize(df):
    scaler = MinMaxScaler(feature_range=(-1, 1))
    dates_scaled = scaler.fit_transform(df)

    return dates_scaled


times_new = normalize(times_dt)
times_new = pd.DataFrame(times_new)
times_new.head()

/Users/myles/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype datetime64[ns] were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


,0,1
0,0.955968,0.955966
1,0.955958,0.955956
2,0.955960,0.955962
3,0.955969,0.955966
4,0.955957,0.955957


In [17]:
times_new.tail()

,0,1
6941019,0.964675,0.964676
6941020,0.964680,0.964679
6941021,0.964682,0.964680
6941022,0.964675,0.964674
6941023,0.964678,0.964677


After converting to datetime objects and normalizing the columns, we can see that there isn't much of a range. This is most likely becuase the year and month were included, placing all the values relatively close to each other. Becuase the tail of the datasaet includes rides that spill over into July, the day as well as the month will need to be included to make it easier on the Algorithm. Also while analyzing the dates, I realized many of the entries do not include dates from June of 2019, so we wil have to replace those.

In [18]:
pickup.str.startswith('200').value_counts()

False    6940973
True          51
Name: tpep_pickup_datetime, dtype: int64

In [19]:
pickup = pickup[pickup.str.startswith('2019')]
dropoff = dropoff[dropoff.str.startswith('2019')]

new_ind = pickup.index

In [20]:
pickup.str.startswith('200').value_counts()

False    6940973
Name: tpep_pickup_datetime, dtype: int64

In [21]:
dur_test = np.array(taxi['duration'])

dur_test = dur_test.reshape(-1,1)

scaler = MinMaxScaler(feature_range=(0, 1))
dur_scaled = scaler.fit_transform(dur_test)

/Users/myles/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype timedelta64[ns] was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [22]:
taxi['duration'] = dur_scaled

So the duration column is solved, however to get a more accuarte picture for the algorithm, I wanted to include only the month and day. While the month may skew the numbers, we have to account for the spill overs into the next month. However thinking about time in the context of Machine Learning can be a little difficult. Initially I wanted to use a min max scaler however the year, and need for it in a datetime object proved to be difficult. While I continue research on this subject, I am going to use the ordinal encoder for certain reasons. 

In [23]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
pick_enc = lb.fit_transform(pickup)
drop_enc = lb.fit_transform(dropoff)
pick_enc

array([   3501,     684,    1304, ..., 2180897, 2179149, 2179956])

In [24]:
taxi = taxi.reindex(new_ind)
taxi.shape

(6940973, 19)

In [25]:
pick_enc = pd.Series(pick_enc)

In [26]:
drop_enc = pd.Series(drop_enc)

In [27]:
taxi.loc[:,'tpep_pickup_datetime'] = pick_enc

In [28]:
taxi.loc[:,'tpep_dropoff_datetime'] = drop_enc

In [29]:
taxi.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,duration
0,1,3501.0,3031.0,1,0.0,1,0,145,145,2,3.0,0.5,0.5,0.00,0.0,0.3,4.30,0.0,0.740208
1,1,684.0,263.0,1,0.0,1,0,262,263,2,2.5,3.0,0.5,0.00,0.0,0.3,6.30,2.5,0.740084
2,1,1304.0,1899.0,1,4.4,1,0,74,7,2,17.5,0.5,0.5,0.00,0.0,0.3,18.80,0.0,0.743404
3,1,3722.0,3198.0,0,0.8,1,0,145,145,2,2.5,1.0,0.5,0.00,0.0,0.3,4.30,0.0,0.740052
4,1,503.0,705.0,1,1.7,1,0,113,148,1,9.5,3.0,0.5,2.65,0.0,0.3,15.95,2.5,0.742148


In [30]:
taxi = taxi.dropna(axis=0)

In [31]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(taxi, test_size=0.2, random_state=42)

    
taxi_prepared = train_set.drop("total_amount", axis=1)
taxi_labels = train_set["total_amount"].copy()

In [32]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(taxi_prepared, taxi_labels)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [33]:
print("Predictions:", lin_reg.predict(taxi_prepared))

Predictions: [16.45372959 70.06998708  9.34692863 ... 31.65787411 11.05814722
 19.57598484]


In [34]:
from sklearn.metrics import mean_squared_error

taxi_predictions = lin_reg.predict(taxi_prepared)
lin_mse = mean_squared_error(taxi_labels, taxi_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

0.3365272349331667

In [35]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(lin_reg, taxi_prepared, taxi_labels,
                         scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-scores)

In [36]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(lin_rmse_scores)

Scores: [0.33791768 0.33559863 0.33611798 0.33660882 0.33759946 0.3384875
 0.33496524 0.33650522 0.33629041 0.33539912]
Mean: 0.33654900682983857
Standard deviation: 0.001082885956006433


So these scores are prety good (as close to 0 as possible), but we shouldn't be too content. Let's look at another model.

In [37]:
from sklearn import linear_model
reg = linear_model.Ridge(alpha=1)
reg.fit(taxi_prepared, taxi_labels) 

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [39]:
taxi_predictions = reg.predict(taxi_prepared)
reg_mse = mean_squared_error(taxi_labels, taxi_predictions)
reg_rmse = np.sqrt(reg_mse)
reg_rmse

0.33652723554281244

While both the models are linear, the goal of this project is to explore the data and what kind of goal we can set for implementation (for now). However, I didn't normalize the model, and there is no pipeline. 

As with most Machine Learning problems, the biggest question is what tools to use, now that I have a better idea of what kind of data and problems will arise, I now also have a better idea of what tools to use. 